# Guided Capstone Step 4. Pre-Processing and Training Data Development

**The Data Science Method**  


1.   Problem Identification 


2.   Data Wrangling 
  
 
3.   Exploratory Data Analysis   

4.   **Pre-processing and Training Data Development**  
 * Create dummy or indicator features for categorical variables
  * Standardize the magnitude of numeric features
  * Split into testing and training datasets
  * Apply scaler to the testing set
5.   Modeling 
  * Fit Models with Training Data Set
  * Review Model Outcomes — Iterate over additional models as needed.
  * Identify the Final Model

6.   Documentation
  * Review the Results
  * Present and share your findings - storytelling
  * Finalize Code 
  * Finalize Documentation

**<font color='teal'> Start by loading the necessary packages as we did in step 3 and printing out our current working directory just to confirm we are in the correct project directory. </font>**

In [89]:
import os
import pandas as pd
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

**<font color='teal'>  Load the csv file you created in step 3, remember it should be saved inside your data subfolder and print the first five rows.</font>**

In [101]:
fileLoc = os.getcwd() + '/Step 2/data/' 
fileName = 'step3_output.csv'
skiData = pd.read_csv(os.path.join(fileLoc, fileName))
skiData.head()

,Name,state,summit_elev,vertical_drop,base_elev,trams,fastEight,fastSixes,fastQuads,quad,...,SkiableTerrain_ac,Snow Making_ac,daysOpenLastYear,yearsOpen,averageSnowfall,AdultWeekday,AdultWeekend,projectedDaysOpen,NightSkiing_ac,clusters
0,Hilltop Ski Area,Alaska,2090,294,1796,0,0.0,0,0,0,...,30.0,30.0,150.000000,36.0,69.0,30.0,34.00000,152.0,30.0,0
1,Sunrise Park Resort,Arizona,11100,1800,9200,0,0.0,0,1,2,...,800.0,80.0,115.000000,49.0,250.0,74.0,78.00000,104.0,80.0,2
2,Yosemite Ski & Snowboard Area,California,7800,600,7200,0,0.0,0,0,0,...,88.0,0.0,110.000000,84.0,300.0,47.0,47.00000,107.0,0.0,1
3,Boreal Mountain Resort,California,7700,500,7200,0,0.0,0,1,1,...,380.0,200.0,150.000000,54.0,400.0,49.0,64.16681,150.0,200.0,1
4,Dodge Ridge,California,8200,1600,6600,0,0.0,0,0,1,...,862.0,0.0,115.103943,69.0,350.0,78.0,78.00000,140.0,0.0,1


## Create dummy features for categorical variables

**<font color='teal'> Create dummy variables for `state`. Add the dummies back to the dataframe and remove the original column for `state`. </font>**

Hint: you can see an example of how to execute this in Aiden's article on preprocessing [here](https://medium.com/@aiden.dataminer/the-data-science-method-dsm-pre-processing-and-training-data-development-fd2d75182967). 

In [102]:
states = skiData['state']
skiDataD = pd.concat([skiData.drop('state', axis=1), pd.get_dummies(states)], axis=1)
skiDataD.head()

,Name,summit_elev,vertical_drop,base_elev,trams,fastEight,fastSixes,fastQuads,quad,triple,...,Rhode Island,South Dakota,Tennessee,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming
0,Hilltop Ski Area,2090,294,1796,0,0.0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,Sunrise Park Resort,11100,1800,9200,0,0.0,0,1,2,3,...,0,0,0,0,0,0,0,0,0,0
2,Yosemite Ski & Snowboard Area,7800,600,7200,0,0.0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,Boreal Mountain Resort,7700,500,7200,0,0.0,0,1,1,3,...,0,0,0,0,0,0,0,0,0,0
4,Dodge Ridge,8200,1600,6600,0,0.0,0,0,1,2,...,0,0,0,0,0,0,0,0,0,0


## Standardize the magnitude of numeric features

**<font color='teal'> Using sklearn preprocessing standardize the scale of the features of the dataframe except the name of the resort which we done't need in the dataframe for modeling, so it can be droppped here as well. Also, we want to hold out our response variable(s) so we can have their true values available for model performance review. Let's set `AdultWeekend` to the y variable as our response for scaling and modeling. Later we will go back and consider the `AdultWeekday`, `dayOpenLastYear`, and `projectedDaysOpen`. For now leave them in the development dataframe. </font>**

In [103]:
# first we import the preprocessing package from the sklearn library
from sklearn import preprocessing

# Declare an explanatory variable, called X,and assign it the result of dropping 'Name' and 'AdultWeekend' from the df
X = skiDataD.drop(['Name','AdultWeekend'], axis=1)

# Declare a response variable, called y, and assign it the AdultWeekend column of the df 
y = skiDataD.AdultWeekend 

# Here we use the StandardScaler() method of the preprocessing package, and then call the fit() method with parameter X 
scaler = preprocessing.StandardScaler().fit(X)

# Declare a variable called X_scaled, and assign it the result of calling the transform() method with parameter X 
X_scaled=scaler.transform(X) 

## Split into training and testing datasets

**<font color='teal'> Using sklearn model selection import train_test_split, and create a 75/25 split with the y = `AdultWeekend`. We will start by using the adult weekend ticket price as our response variable for modeling.</font>**

In [104]:
# Import the train_test_split function from the sklearn.model_selection utility.  
from sklearn.model_selection import train_test_split

# Get the 1-dimensional flattened array of our response variable y by calling the ravel() function on y
y = y.ravel()

# Call the train_test_split() function with the first two parameters set to X_scaled and y 
# Declare four variables, X_train, X_test, y_train and y_test separated by commas 
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.25, random_state=1)

Here we start the actual modeling work. First let's fit a multiple linear regression model to predict the `AdultWeekend` price.

# Guided Capstone Step 5. Modeling


This is the fifth step in the Data Science Method. In the previous steps you cleaned and prepared the datasets. Now it's time to get into the most exciting part: modeling! In this exercise, you'll build three different models and compare each model's performance. In the end, you'll choose the best model for demonstrating insights to Big Mountain management.



### **The Data Science Method**  


1.   Problem Identification 

2.   Data Wrangling 
  
3.   Exploratory Data Analysis 
 
4.   Pre-processing and Training Data Development

5.   **Modeling**
  * Fit Models with Training Data Set
  * Review Model Outcomes — Iterate over additional models as needed.
  * Identify the Final Model

6.   Documentation
  * Review the Results
  * Present and share your findings - storytelling
  * Finalize Code 
  * Finalize Documentation

## Fit Models with a Training Dataset

**<font color='teal'> Using sklearn, fit the model on your training dataset.</font>**

#### Model 1

In [105]:
#all first model set
from sklearn import linear_model
from sklearn.metrics import explained_variance_score,mean_absolute_error
lm = linear_model.LinearRegression()
model1 = lm.fit(X_train,y_train)

**<font color='teal'> Predict on the testing dataset and score the model performance with the y_test set and the y-pred values. The explained variance is a measure of the variation explained by the model. This is also known as the R-squared value. </font>**

Hint: you will have to use the `predict()` method here as it's used in this [DSM article](https://medium.com/@aiden.dataminer/the-data-science-method-dsm-modeling-56b4233cad1b) about modeling. 

In [106]:
# Make a variable called y_pred and assign it the result of calling predict() on our model variable with parameter X_test
y_pred = model1.predict(X_test)

## Review Model Outcomes — Iterate over additional models as needed

In [107]:
# You might want to use the explained_variance_score() and mean_absolute_error() metrics.
# To do so, you will need to import them from sklearn.metrics. 
# You can plug y_test and y_pred into the functions to evaluate the model

exVarScoreM1 = explained_variance_score(y_test, y_pred)
MAEM1 = mean_absolute_error(y_test, y_pred)


In [108]:
print('Model 1 predicted ' + str(exVarScoreM1*100) + '% of the variance.') 
print('Model 1 has a mean squared error of: ' + str(MAEM1))

Model 1 predicted 68.52758764805125% of the variance.
Model 1 has a mean squared error of: 7.030806883619508


**<font color='teal'> Print the intercept value from the linear model. </font>**

Hint: our linear regression model `lm` has an attribute `intercept_` for the intercept

In [109]:
print(lm.intercept_)

55.90574163409337


**<font color='teal'> The intercept is the mean `AdultWeekend` price for all the resorts given the other characteristics. The addition or subtraction of each of the coefficient values in the regression are numeric adjustments applied to the intercept to provide a particular observation's value for the resulting `AdultWeekend` value. Also, because we took the time to scale our x values in the training data, we can compare each of the coeeficients for the features to determine the feature importances. Print the coefficient values from the linear model and sort in descending order to identify the top ten most important features.</font>** 


Hint: make sure to review the absolute value of the coefficients, because the adjustment may be positive or negative, but what we are looking for is the magnitude of impact on our response variable.

In [110]:
# You might want to make a pandas DataFrame displaying the coefficients for each state like so: 
CoefM1 = pd.DataFrame(abs(lm.coef_), X.columns, columns=['Coefficient'])
CoefM1.abs().sort_values(by = 'Coefficient',ascending=False)


,Coefficient
total_chairs,5.690379e+12
trams,5.301745e+12
fastSixes,3.207700e+12
double,3.062774e+12
fastEight,2.968813e+12
surface,2.921758e+12
New York,2.646888e+12
triple,2.598026e+12
Michigan,2.328488e+12
California,2.101858e+12


**<font color='teal'>You should see that the top ten important features are different states. However, the state is not something the managers at the Big Mountain Resort can do anything about. Given that we care more about actionable traits associated with ticket pricing, rebuild the model without the state features and compare the results. </font>**

Hint: Try to construct another model using exactly the steps we followed above. 

#### Model 2

In [111]:
#We will need to build new training and test data to feed into the model to drop the state column from the data set. 

#Start by reloading the data:

from sklearn import preprocessing
# Declare an explanatory variable, called X,and assign it the result of dropping 'Name' and 'AdultWeekend' from the df
XnS = skiData.drop(['Name', 'AdultWeekend','state'], axis=1)

# Declare a response variable, called y, and assign it the AdultWeekend column of the df 
ynS = skiData.AdultWeekend 

# Here we use the StandardScaler() method of the preprocessing package, and then call the fit() method with parameter X 
scaler = preprocessing.StandardScaler().fit(XnS)

# Declare a variable called X_scaled, and assign it the result of calling the transform() method with parameter X 
XnS_scaled=scaler.transform(XnS) 

from sklearn.model_selection import train_test_split

# Get the 1-dimensional flattened array of our response variable y by calling the ravel() function on y
ynS = y.ravel()

# Call the train_test_split() function with the first two parameters set to X_scaled and y 
# Declare four variables, X_train, X_test, y_train and y_test separated by commas 
XnS_train, XnS_test, ynS_train, ynS_test = train_test_split(XnS_scaled, ynS, test_size=0.25, random_state=1)

# Build the model2
from sklearn import linear_model
from sklearn.metrics import explained_variance_score,mean_absolute_error
lm = linear_model.LinearRegression()
model2 = lm.fit(XnS_train,ynS_train)

# Make a variable called y_pred and assign it the result of calling predict() on our model variable with parameter X_test
ynS_pred = model2.predict(XnS_test)

exVarScoreM2 = explained_variance_score(ynS_test, ynS_pred)
MAEM2 = mean_absolute_error(ynS_test, ynS_pred)

print('Model 2 predicted ' + str(exVarScoreM2*100) + '% of the variance.') 
print('Model 2 has a mean squared error of: ' + str(MAEM2))

Model 2 predicted 65.80619023724286% of the variance.
Model 2 has a mean squared error of: 7.48950636214842


In [112]:
CoefM2 = pd.DataFrame(abs(lm.coef_), XnS.columns, columns=['Coefficient'])
CoefM2.abs().sort_values(by = 'Coefficient',ascending=False)

,Coefficient
AdultWeekday,1.018778e+01
summit_elev,6.865626e+00
base_elev,6.348884e+00
SkiableTerrain_ac,2.156880e+00
daysOpenLastYear,2.038537e+00
quad,1.813407e+00
projectedDaysOpen,1.673020e+00
surface,1.643747e+00
clusters,1.298535e+00
vertical_drop,1.278570e+00


**<font color='teal'> When reviewing our new model coefficients, we see `summit_elev` is now in the number two spot. This is also difficult to change from a management prespective and highly correlated with `base_elev` and `vertical_drop`.  This time, rebuild the model without the state features and without the `summit_elev` and without `base_elev`and compare the results. </font>**

#### Model 3

In [115]:
#We will need to build new training and test data to feed into the model to drop the state column from the data set. 

#Start by reloading the data:

from sklearn import preprocessing
# Declare an explanatory variable, called X,and assign it the result of dropping 'Name' and 'AdultWeekend' from the df
XnE = XnS.drop(['summit_elev'], axis=1)

# Declare a response variable, called y, and assign it the AdultWeekend column of the df 
ynE = skiData.AdultWeekend 

# Here we use the StandardScaler() method of the preprocessing package, and then call the fit() method with parameter X 
scaler = preprocessing.StandardScaler().fit(XnE)

# Declare a variable called X_scaled, and assign it the result of calling the transform() method with parameter X 
XnE_scaled=scaler.transform(XnE) 

from sklearn.model_selection import train_test_split

# Get the 1-dimensional flattened array of our response variable y by calling the ravel() function on y
ynE = y.ravel()

# Call the train_test_split() function with the first two parameters set to X_scaled and y 
# Declare four variables, X_train, X_test, y_train and y_test separated by commas 
XnE_train, XnE_test, ynE_train, ynE_test = train_test_split(XnE_scaled, ynE, test_size=0.25, random_state=1)



In [119]:
# Build the model3
from sklearn import linear_model
from sklearn.metrics import explained_variance_score,mean_absolute_error
lm = linear_model.LinearRegression()
model3 = lm.fit(XnE_train,ynE_train)

# Make a variable called y_pred and assign it the result of calling predict() on our model variable with parameter X_test
ynE_pred = model3.predict(XnE_test)

exVarScoreM3 = explained_variance_score(ynE_test, ynE_pred)
MAEM3 = mean_absolute_error(ynE_test, ynE_pred)

print('Model 3 predicted ' + str(exVarScoreM3*100) + '% of the variance.') 
print('Model 3 has a mean squared error of: ' + str(MAEM3))

Model 3 predicted 65.77351527413407% of the variance.
Model 3 has a mean squared error of: 7.493961628510009


In [122]:
CoefM3 = pd.DataFrame(abs(lm.coef_), XnE.columns, columns=['Coefficient'])
CoefM3.abs().sort_values(by = 'Coefficient',ascending=False)

,Coefficient
AdultWeekday,1.023764e+01
vertical_drop,2.499936e+00
SkiableTerrain_ac,2.154279e+00
daysOpenLastYear,2.046571e+00
quad,1.877412e+00
surface,1.694838e+00
projectedDaysOpen,1.621243e+00
LongestRun_mi,1.174463e+00
TerrainParks,1.168130e+00
clusters,1.145934e+00


## Identify the Final Model

**<font color='teal'> Review the model performances in the table below and choose the best model for proving insights to Big Mountain management about what features are driving ski resort lift ticket prices. Type your choice in the final markdown cell — you will discuss this selection more in the next step of the guided casptone. </font>**

| Model | Explained Variance| Mean Absolute Error|Features Dropped|
| --- | --- | --- | --- |
| Model 1. | 0.685276 | 7.0308 |-|
| Model 2. | 0.658062 | 7.4895 |'state'|
| Model 3. | 0.657735 | 7.4939 |'state','summit_elev','base_elev'|

Model Selection: Model 3 because it has the lowest MAE of the models that do not contain irrelevant features